In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.corpus import stopwords
from nltk import FreqDist, bigrams, trigrams,ngrams
from nltk import WordNetLemmatizer
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
# import gensim
# from gensim.models.keyedvectors import KeyedVectors
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, LSTM, GRU
from keras.regularizers import l1, l2
from sklearn.metrics import classification_report
import collections
import warnings
warnings.filterwarnings("ignore")
zomato = pd.read_csv("zomato.csv", na_values = ["-", ""])
# Making a copy of the data to work on
data = zomato.copy()
data.drop(['url','address','menu_item','phone','location'],axis=1,inplace=True)
data.rename(columns={'listed_in(city)':'location'}, inplace=True)
data=data[data['rate'].notnull()]

Using TensorFlow backend.


In [2]:
data["cuisines"] = data["cuisines"].astype(str)

In [3]:
sam=data[['cuisines','location']].groupby(['location'],as_index=False).agg(lambda x:' '.join(list(x)))

In [4]:
pair={}
for i in range(0,30):
    s=sam[sam.index==i]
    s["cuisines"]= s["cuisines"].apply(lambda x: x.lower())
    tokenizer = regextoken("[a-zA-Z]+") 
    cuisine_tokens = s["cuisines"].apply(tokenizer.tokenize)
    all_cuisines = cuisine_tokens.astype(str).str.cat()
    cleaned_cuisines = tokenizer.tokenize(all_cuisines)
    j=s['location'].values[0]
    pair[j] = cleaned_cuisines

In [5]:
tree={}

In [6]:
for i in pair.keys():
    trigram = lambda a: zip(a, a[1:],a[2:])
    ap=list(trigram(pair[i]))
    trigramFreq = collections.Counter(ap)
    tree[i]=trigramFreq.most_common(10)

In [7]:
too={}
for i in pair.keys():    
    bigram = lambda a: zip(a, a[1:])
    apw=list(bigram(pair[i]))
    bigramFreq = collections.Counter(apw)
    too[i]=bigramFreq.most_common(10)

In [9]:
tree.keys()

dict_keys(['BTM', 'Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur', 'Brigade Road', 'Brookefield', 'Church Street', 'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar', 'JP Nagar', 'Jayanagar', 'Kalyan Nagar', 'Kammanahalli', 'Koramangala 4th Block', 'Koramangala 5th Block', 'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road', 'MG Road', 'Malleshwaram', 'Marathahalli', 'New BEL Road', 'Old Airport Road', 'Rajajinagar', 'Residency Road', 'Sarjapur Road', 'Whitefield'])

In [8]:
sorted(too['Banashankari'])

[(('chinese', 'north'), 44),
 (('fast', 'food'), 162),
 (('food', 'south'), 35),
 (('indian', 'chinese'), 166),
 (('indian', 'fast'), 35),
 (('indian', 'north'), 99),
 (('indian', 'south'), 65),
 (('north', 'indian'), 286),
 (('south', 'indian'), 227),
 (('street', 'food'), 72)]